In [1]:
import seaborn as sns
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
sns.set_style("whitegrid")

from attrdict import AttrDict
from drivers import *
from benchmarks import jobs


/Users/lguerdan/.virtualenvs/10701-Project/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
exp_config = AttrDict({
    'benchmark': {
        'name': 'synthetic_1D_sinusoidal',
        'NS': 50000,
        'config': {
            'Y0_PDF': 'piecewise_sinusoid',
            'Y1_PDF': 'low_base_rate_sinusoid',
            'PI_PDF': 'linear'
            }
    },
    
    'n_epochs': 5,
    
    # Experimental setup 
    'learn_weights': True,
    'learn_parameters': True,
    
    # Hyperparameters
    'n_epochs': 20,
    'n_runs': 2,
    
    'target_POs': [0],
    
    'policy_gamma': 0,
    'train_ratio': .7,
    
    'split_erm': True,
    'crossfit_erm': True,
    'split_ccpe': True,
    'crossfit_ccpe': True,
})

error_params = [AttrDict({
    'alpha_0': 0.5,
    'alpha_1': 0,
    'beta_0': 0.1,
    'beta_1': 0 
})]

baselines = [AttrDict({
    'model': 'OBS',
    'target': 'Y',
    'conditional': False,
    'sl': False,
    'reweight': False
}), AttrDict({
    'model': 'OBS Oracle',
    'target': 'YS',
    'conditional': False,
    'sl': False,
    'reweight': False
}), AttrDict({
    'model': 'COM',
    'target': 'Y',
    'conditional': True,
    'sl': False,
    'reweight': False
}), AttrDict({
    'model': 'COM-SL',
    'target': 'Y',
    'conditional': True,
    'sl': True,
    'reweight': False
}), AttrDict({
    'model': 'RW',
    'target': 'Y',
    'conditional': True,
    'sl': False,
    'reweight': True
}), AttrDict({
    'model': 'RW-SL',
    'target': 'Y',
    'conditional': True,
    'sl': True,
    'reweight': True
}), AttrDict({
    'model': 'Proxy Oracle',
    'target': f'Y',
    'conditional': True,
    'sl': False,
    'reweight': False
}), AttrDict({
    'model': 'Target Oracle',
    'target': f'YS',
    'conditional': True,
    'sl': True,
    'reweight': False
})]


crossfit_preds, Y_test, config =  run_risk_minimization_exp(exp_config, baselines, error_params)



AttrDict({'alpha_0': 0.5, 'alpha_1': 0, 'beta_0': 0.1, 'beta_1': 0})
RUN: 0, alpha_0: 0.5, alpha_1: 0, beta_0: 0.1, beta_1: 0

AttrDict({'alpha_0': 0.5, 'alpha_1': 0, 'beta_0': 0.1, 'beta_1': 0})


ERM: OBS Oracle:  70%|██████████████████▉        | 7/10 [00:05<00:02,  1.38it/s]Traceback (most recent call last):
  File "<frozen importlib._bootstrap>", line 203, in _lock_unlock_module
KeyboardInterrupt

Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/usr/local/Cellar/python@3.9/3.9.8/Frameworks/Python.framework/Versions/3.9/lib/python3.9/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/usr/local/Cellar/python@3.9/3.9.8/Frameworks/Python.framework/Versions/3.9/lib/python3.9/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
  File "/Users/lguerdan/.virtualenvs/10701-Project/lib/python3.9/site-packages/torch/__init__.py", line 721, in <module>
    _C._initExtension(manager_path())
  File "/Users/lguerdan/.virtualenvs/10701-Project/lib/python3.9/site-packages/torch/cuda/__init__.py", line 759, in <module>
    from . import amp
  File "/Users/lguerdan/.virtua

RuntimeError: DataLoader worker (pid 54133) exited unexpectedly with exit code 1. Details are lost due to multiprocessing. Rerunning with num_workers=0 may give better error trace.

In [30]:

n_splits = 3  # (function argument)
te_metrics, po_metrics = compute_crossfit_metrics(crossfit_preds, Y_test, n_splits, config)

In [33]:
pd.DataFrame(po_metrics)

,AU-ROC,ACC,do,baseline
0,0.400174,0.0,0,OBS
1,0.447338,0.0,0,OBS Oracle
2,0.540643,0.0,0,COM
3,0.513889,0.0,0,COM-SL
4,0.384615,0.0,0,RW
5,0.482060,0.0,0,RW-SL
6,0.529425,0.0,0,Proxy Oracle
7,0.615385,0.0,0,Target Oracle


In [ ]:
X, Y = load_benchmark(exp_config.benchmark, error_params)

split_ix = int(.7*X.shape[0])

ccpe_dataset = AttrDict({
    'X_train': X.iloc[split_ix:, :],
    'Y_train': Y.iloc[split_ix:, :],
    'X_test': X.iloc[:split_ix, :],
    'Y_test': Y.iloc[:split_ix, :],
})



In [12]:
Y['pYS_0'].min()

1.94447574708434e-09

In [21]:
error_params

{'alpha_0': 0.1, 'alpha_1': 0.05, 'beta_0': 0.2, 'beta_1': 0.4}

In [6]:
py, alpha, beta = ccpe(ccpe_dataset, 0, exp_config, q_max=0, q_min=1)

Conditioning


Target: Y: 100%|████████████████████████████████| 20/20 [00:19<00:00,  1.01it/s]


In [7]:
alpha

0.97248465

In [8]:
beta

0.7411392331123352

In [6]:
beta

0.9778537675738335

In [10]:

np.quantile(py_hat, 0)


py.min()

0.28198478

In [12]:
train_ratio = .7
N = X.shape[0]

test_split_ix = int(N*train_ratio)

X_train, X_test = X.iloc[split_ix:, :], X.iloc[:split_ix, :]
Y_train, Y_test = Y.iloc[split_ix:, :], Y.iloc[:split_ix, :]

weight_split_ix

ccpe_train_split_ix


In [14]:
X_train.shape

(1500, 2)

In [6]:
X.shape

(5000, 1)

In [10]:
X.shape[1]

2